# Grad Project # 2

In this project, continuing the "search and rescue" domain from the first project, in this assignment, we will consider the problem of inferring the location of the robot as it moves around. Specifically, you will:

* Implement a particle filter for the localization problem, to allow the robot to estimate its position. 
* Implement a particle filter for the simultaneous localization and mapping (SLAM) problem, to allow the robot both to estimate its position and the map of its environment.  
* Run simulations to test the performance of your algorithms.

Your project will consist of implementing a particle filter, and also performing experiments and analysis on the different implementations. Your project submission will consist of the code you implement, a PDF of your experiments and movies we've asked you to generate.

**The tests in this notebook are not part of your grade**. They are only to help you sanity check your implementation. Your project grade will be based on the report, figures, animation, and notebook as a whole. 

In [ ]:
# Setup matplotlib animation
import matplotlib
matplotlib.rc('animation', html='jshtml')


## Imports and Utilities

These are import and utility functions, and also scaffolding of functions that we have provided for you for the project.

In [ ]:
from typing import Optional


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation for a class method.

    This utility allows one to define methods outside of a class defintion.
    Thus allowing one to provide the implementations of a class "incrementally".
    This utility is useful for gradually submitting class method implementations
    to catsoop.

    It is intended to be used as a decorator.

    Args:
        cls: a Python class on which we are implementing the method.
        method_name: an optional method name to indicate which method we are
            implementing. If not provided, infer the method name by using the
            decorated function's name.

    Example:
        >>> class A:
        ...   def foo(self):
        ...     raise NotImplementedError()

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator



from typing import (Callable, Iterable, List, Sequence, Optional, Dict)

from abc import abstractmethod, ABCMeta
import itertools
import math
import random
import dataclasses
from collections import namedtuple
import copy

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (str) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])
# delta_p --- distance to move; delta_theta --- rotation of the robot
Command = namedtuple("Command", ["delta_p", "delta_theta"])


def grid_of_landmarks(
        x_range: Iterable[float] = range(-4, 20, 4),
        y_range: Iterable[float] = range(-4, 20, 4),
) -> Sequence[Landmark]:
    """Constructs a grid of landmarks from a catesion product of x and y coordinates."""
    return tuple(
        Landmark(f"landmark-{i}", *loc)
        for i, loc in enumerate(itertools.product(x_range, y_range)))


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM variants)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        """Returns the current estimation of the robot's Pose."""
        ...

    @abstractmethod
    def estimated_landmarks(self) -> Sequence[Landmark]:
        """Returns the current estimation of the landmarks."""
        ...

    @abstractmethod
    def init(self, init_state: Pose) -> None:
        """Initialize the inference with an initial pose."""
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        """An update step of the inference, called after every simulation step."""
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        """Helper to visualize the internal state of the inference algorithm."""
        ...


def normalize_angles(angles: np.array) -> np.ndarray:
    """Given an array of angles in radians, element-wise normalize the angles to the range [-pi, pi].

    Args:
        angles: array of any shape.

    Returns:
        normalized angles, array of same shape as the input.
    """
    return np.arctan2(np.sin(angles), np.cos(angles))


def circular_mean(angles: np.array, axis=None) -> float:
    """Given an array of angles in radians, find the [circular mean](https://en.wikipedia.org/wiki/Circular_mean).

    Args:
        angles: array of any shape.
        axis: axis or axes along which the means are computed. The default is to compute the mean of the flattened array.

    Returns:
        mean of the angles, normalized to [-pi, pi].
    """
    return np.arctan2(np.sum(np.sin(angles), axis=axis),
                      np.sum(np.cos(angles), axis=axis))


@dataclasses.dataclass(frozen=True)
class SimulationResult:
    """A helper class to hold the results of running a simulation."""
    sim: 'Simulator'  # Simulator that generated this result
    infer: Optional[Inference]  # Inference procedure that generated this result
    true_poses: Sequence[Pose]
    estimated_poses: Optional[Sequence[Pose]] = None
    # History snapshots for the Inference, for animation purpose
    snapshots: Optional[Sequence[Inference]] = None


@dataclasses.dataclass(frozen=False)
class Simulator:
    """A simulator for a point robot in R^2.

    The simulator simulates the motion and sensor noises of the robot by
    sampling from 2D Gaussians.
    The simulation has a list of landmarks, each with a unique identifier.
    The sensor of the robot can sense, for each landmark in the robot's sensing
    range, a noise-corrupted distance and bearing to that landmarks (together
    with the identifier that identifies the landmark).
    """

    # Initial simulator state
    init_state: Pose = Pose(0, 0, 0)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.diag([1e-3, np.deg2rad(3)**2])

    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)**2])

    # A set of landmarks
    landmarks: Sequence[Landmark] = grid_of_landmarks()

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    # Random seed control
    rng: np.random.Generator = dataclasses.field(init=False, default=None)
    # Using the same seed will induce the same trajectory
    seed: dataclasses.InitVar[int] = 0

    def __post_init__(self, seed: int):
        assert (len({l.id for l in self.landmarks
                    }) == len(self.landmarks)), "Landmark must have unique IDs"
        self.rng = np.random.default_rng(seed)

    def init(self) -> None:
        self.state = self.init_state

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the
          distance of the movement, and delta_theta(float), the rotation of
          the movement.
        """
        noise = self.rng.multivariate_normal(np.zeros(2),
                                             self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        # Simulate that the measurements' order is random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta
            # Normalize angle to -pi to pi
            b = math.atan2(math.sin(b), math.cos(b))

            noise = self.rng.multivariate_normal(np.zeros(2),
                                                 self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[Inference] = None,
            snapshot_every_n: Optional[int] = 0) -> SimulationResult:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot.

        Returns:
            SimulationResult
        """
        true_poses, est_poses, snapshots = [], None, None

        self.init()

        i = 0
        true_poses.append(self.state)
        if infer:
            est_poses, snapshots = [], []
            infer.init(self.init_state)
            est_poses.append(infer.estimated_pose())
            if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                snapshots.append(copy.deepcopy(infer))

        for command in commands:
            i += 1
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return SimulationResult(self, copy.deepcopy(infer), true_poses,
                                est_poses, snapshots)


def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def patrol_commands() -> Iterable[Command]:
    """Commands that walks back and forth horizontally."""
    for i in range(10):
        yield Command(0.5, 0)
    yield Command(0, np.pi)





def run_inference(sim: Simulator,
                  commands: Sequence[Command] = tuple(
                      drive_in_a_square_commands()),
                  mode: str = "localization",
                  num_particles: int = 100,
                  snapshot_every_n=0) -> SimulationResult:
    """Utility to run inference on a simulator with a particular inference mode.

    Args:
        sim: the simulator instance to run the simulation.
        commands: see `Simulator.run`.
        mode: specified the mode of the inference; one of "localization", "SLAM" or "RBSLAM".
            You must implement the corresponding inference algorithm in order
            to perform inference in any mode.
        num_particles: number of particles of the particle filter.
        snapshot_every_n: see `Simulator.run`.

    Returns:
        A simulation result returned by `Simulator.run`.
    """

    mode = mode.lower()
    try:
        if mode == "localization":
            infer = Localization(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                landmarks=sim.landmarks,
                num_particles=num_particles,
            )
        elif mode == "slam":
            infer = SLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        elif mode == "rbslam":
            infer = RBSLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        else:
            raise ValueError(f"Unrecoginized inference: {mode}")
    except NotImplementedError:
        raise NotImplementedError(
            f"You must completely implement the {infer.__class__} class to "
            f"run simulation under {mode} mode")

    return sim.run(commands, infer=infer, snapshot_every_n=snapshot_every_n)


def plot_samples(x, y, fov=((-6, -6), (20, 20)), ax=None, show=True):
    """Plots a list of samples of x, y coordinates as input and scatter plot the samples.

    Args:
        x: array of shape (N, ), the x-coordinates of the pose component of the particles.
        y: array of shape (N, ), the y-coordinates of the pose component of the particles.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(x, y, 'g+')

    if show:
        plt.show()

    return ax


def plot_trajectories(true_poses: Sequence[Pose],
                      estimated_poses: Sequence[Pose],
                      landmarks: Optional[Sequence[Landmark]] = None,
                      ax: Optional['matplotlib.axes.Axes'] = None,
                      fov=((-6, -6), (20, 20)),
                      show=True):
    """Plot true and estimated poses on the same plot by overlaying them.
    It ignores the pose.theta and only plots the location.

    Args:
        true_poses: a sequence of true poses.
        estimated_poses: a sequence of estimated poses.
        landmarks: optinally plot the location of true landmarks.
        ax: an optional matplotlib Axes on which to plot the trajectories.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    if landmarks:
        ax.scatter([l.x for l in landmarks], [l.y for l in landmarks],
                   marker="o",
                   color="black")

    ax.plot([p.x for p in true_poses], [p.y for p in true_poses],
            color=(0, 0, 1, 0.5))
    ax.plot([p.x for p in estimated_poses], [p.y for p in estimated_poses],
            color=(1, 0, 0, 0.5))

    if show:
        plt.show()
    return ax


def plot_simulation_result(result: SimulationResult,
                           ax: Optional['matplotlib.axes.Axes'] = None,
                           fov=((-6, -6), (20, 20)),
                           title=None,
                           show=True):
    """Plot the inference result (the state at last step of the simulation result).

    Args:
        result: result of a simulation.
        ax: an optional matplotlin Axes to draw the result.
        fov: see `plot_trajectories`.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    if title:
        ax.set_title(title)

    plot_trajectories(result.true_poses,
                      result.estimated_poses,
                      landmarks=result.sim.landmarks,
                      ax=ax,
                      fov=fov,
                      show=False)

    result.infer.plot_state(ax)

    if show:
        plt.show()

    return ax


def render_animation(
    result: SimulationResult,
    fov=((-6, -6), (20, 20)),
    save_file: Optional[str] = None,
) -> 'matplotlib.animation.FuncAnimation':
    """Render simulation as a matplotlib animation.

    Args:
        result: result of a simulation.
        fov: see `plot_trajectories`.
        save_file: an optional string file name to save the animation.

    Returns:
        an instance of `matplotlib.animation.FuncAnimation`. You can do further
        processing on the returned animation, display it, or save it
        as an mp4 file.
    """
    if result.snapshots is None or len(result.snapshots)==0:
        raise ValueError("SimulationResult does not have any snapshots. "
                         "Please re-run simulation with snapshot_every_n > 0.")

    import matplotlib.animation as animation
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    snapshot_every_n = len(result.true_poses) // len(result.snapshots)
    true_poses = result.true_poses[::snapshot_every_n]
    estimated_poses = result.estimated_poses[::snapshot_every_n]

    def render_frame(i):
        ax.clear()
        ax.set_title(f"Step {i}")
        plot_trajectories(true_poses[:i + 1],
                          estimated_poses[:i + 1],
                          landmarks=result.sim.landmarks,
                          ax=ax,
                          fov=fov,
                          show=False)
        result.snapshots[i].plot_state(ax)
        return ax

    anim = animation.FuncAnimation(fig, render_frame, len(result.snapshots))
    if save_file:
        anim.save(f"{save_file}.mp4", fps=60)
    return anim





# 1. Warming up with sampling

Let us first check to make sure we know how to sample from different distributions.

### 1.1 Warmup 1

Let's make sure we know how to sample from a set
of values based on their weights.

Please write a function that:
- Takes in a list of samples and associated weights.
- Re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

_Hint:_ You should use the function `numpy.random.choice`.


For reference, our solution is **1** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray,
             nr_samples: int) -> np.ndarray:
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

### 1.2 Warmup 2

We are going to be sampling from Gaussian distributions in this project. While we have not talked about continuous random variables and continuous probability densities much during the lectures, we can use the ideas from Monte Carlo inference techniques to handle continuous variables quite easily. 

First, let us check that we know how to sample from a multivariate Gaussian.
Please write a function that takes in a mean vector, a covariance matrix and a integer number
of samples and returns a list of the requested number of samples drawn from a Gaussian with the given mean and covariance.

_Hint:_ You should use the function `numpy.random.multivariate_normal`.

For reference, our solution is **1** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.

    Returns:
      sample: a numpy vector with length num_particles.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### 1.3 Experiment 1: Sampling from Multivariate Gaussian

Please generate a 2D scatter plot of 1000 samples drawn from a 2D Gaussian with `mean = [0, 0]` and `covariance = np.diag([0.05, 0.01])`, using the function you just coded: `warmup_2`. Consider using the helper function provided: `plot_samples`.

<b>Submission Material 1a:</b> In your submitted pdf, please include the generated figure. Name this Figure 1a.

Let's get some intuition about geometry. Please generate a second 2D scatter plot of 1000 samples drawn from a 2D Gaussian distribution. You should pick the mean and covariance so that the figure shows an ellipse with a $45$-degree angle with the x-y axes. Specifically, try to modify the mean and covariance to rotate figure 1a by 45-degrees counter-clockwise.


<b>Submission Material 1b:</b> In your submitted pdf, please include the generated figure. Name this Figure 1b.


### 1.4 Experiment 2: Motion Noise 

We have provided a simulator class for you, that simulates the noisy motion
and sensing of the robot. Let's begin by ignoring any observations and just trusting that the robot goes where we have commanded. We'll look at the relationship between commands and actual trajectories.

We have provided a `Simulator` class for you. It simulates the noisy motion and sensing of the robot. Let's begin by exploring what happens if we don't do any inference, and assume we always know where the robot is.

We have provided a method `Simulator.run` that simulates the motion of the robot given a sequence of commands. For now, you may ignore the `infer` parameter. If you call this method, you will get a `SimulationResult`. Here, `SimulationResult.true_poses` holds a list of poses representing the ''true'' trajectory of the robot. You can ignore the rest of the fields in `SimulationResult` for now.

Please call `run` twice, passing in `drive_in_a_square_commands()` as the commands.
- For the first time, run on a simulator with `motion_noise_covariance=zeros((2, 2))`
- For the second time, run on a simulator with `motion_noise_covariance=np.diag([1e-3, np.deg2rad(3)**2])`. 

The `drive_in_a_square_commands()` generates a sequence of commands that attempt to drive the robot in a square motion.
However, you should see a significant difference between the trajectories without noise, which should be a perfect square, versus with noise. 

**Hint**: You may use the helper function `plot_trajectories` to generate the trajectory plot.

<b>Submission Material 2:</b> In your submitted pdf, please include a single figure which has both experiment results overlapped on each other. Name this Figure 2.
If we knew the noise on each step, the noise wouldn't be a problem. We could just correct for it as we steer the robot, but the problem is we don't the noise --- it's just the random perturbations added by nature. Therefore, we have to use the sensors to know where the robot is and correct for the "drift" in the robot motion. **That** is why we perform inference.

Now we've convinced ourselves of the need for inference, let us implement the two functions we need for sampling-based inference.

## 2. Particle Filter Localization 

Now, let’s assume that there are some landmarks (e.g. radio beacons or visible mountains in the distance) with known absolute locations. The robot can observe measurements of its relative position to these landmarks. This problem becomes an HMM, in which the hidden state is the robot’s position and orientation (called a "pose" for short), the noisy steering we saw above is the transition model, and the landmark measurements are the observations. We are interested in performing filtering to obtain a distribution over the robot’s pose at each time step. 

We could discretise the robot position, and perform HMM inference as we saw in class, but discretising the states is going to lead to a very large discrete state space that will be hard to perform inference efficiently in. If you have taken a robotics course or state estimation course elsewhere, you might be familiar with the Kalman filter as a way to estimate the state of a linear system with Gaussian noise, but as we will see, the transition dynamics of our robot are substantially non-linear and a Kalman filter will not perform well.

Instead, we will use a particular form of sampling over HMMs known as the particle filter, which we discussed briefly at the very end of the HMM lecture. 

### Utilities

## Localization: Update by Motion Model


In [ ]:
from functools import cached_property

def multivariate_normal_logpdf(xs: np.ndarray, means: np.ndarray,
                               covs: np.ndarray) -> np.ndarray:
    """Batch version of `scipy.stats.multivariate_normal.logpdf`.

    Compute multivariate normal's log probability density function, with batch
    inputs and distributions.
    Modified from https://gregorygundersen.com/blog/2020/12/12/group-multivariate-normal-pdf/.

    This function uses the following broadcasting logic:
    - First, `means` and `covs` are broadcasted together, which determines `dist_batch`,
        the number of individual multivariate normal distributions.
    - Then, broadcast the rightmost dimensions of `xs` to `means`. The rest of the dimensions
    of `xs` becomes the `input_batch` dimension.

    Args:
        xs: array of shape (...input_batch,  ...dist_batch, dist_dim)
        means: array of shape (...dist_batch, dist_dim)
        covs: array of shape (...dist_batch, dist_dim, dist_dim)

    Returns:
        array of shape (...input_batch, ...dist_batch, dist_dim)


    Below we describe some uses cases, with different input shapes and the result shape.

    1.  xs: (100, 2), means: (2,),  covs: (2, 2), result: (100, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=(100,)` and `dist_batch=()`
            - returns logpdf of 100 inputs, evaluated by the same mean and covariance.

    2.  xs: (100, 2), means: (2,),  covs: (100, 2, 2), result: (100, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100,)`
            - returns logpdf of 100 inputs, evaluated by the same mean but with 100 different covariances.

    3.  xs: (100, 10, 2), means: (2,),  covs: (100, 10, 2, 2), result: (100, 10, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100, 10)`
            - returns logpdf of inputs of shape (100, 10), evaluated by the same mean each with a different covariance.
    """
    dist_dim = xs.shape[-1]
    dist_batch_shape = np.broadcast_shapes(means.shape[:-1], covs.shape[:-2])
    means = np.broadcast_to(means, dist_batch_shape + (dist_dim,))
    covs = np.broadcast_to(covs, dist_batch_shape + (dist_dim, dist_dim))
    xs = np.broadcast_to(xs, xs.shape[:-len(means.shape)] + means.shape)

    vals, vecs = np.linalg.eigh(covs)
    logdets = np.sum(np.log(vals), axis=-1)
    valsinvs = 1. / vals
    Us = vecs * np.sqrt(valsinvs)[..., np.newaxis, :]
    devs = xs - means
    devUs = np.einsum("...i,...ij->...j", devs, Us)
    mahas = np.sum(np.square(devUs), axis=-1)
    log2pi = np.log(2 * np.pi)
    out = -0.5 * (dist_dim * log2pi + mahas + logdets)
    return out


@dataclasses.dataclass(frozen=True)
class LocalizationParticles:
    """A batch of particles."""

    x: np.ndarray  # shape (N, )
    y: np.ndarray  # shape (N, )
    theta: np.ndarray  # shape (N, )

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1
        # Always normalize the angles
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class Localization(Inference):
    """Localization Inference by particle filtering."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark] = dataclasses.field(default_factory=list)

    num_particles: int = 10

    particles: LocalizationParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    @cached_property
    def landmarks_id_map(self) -> Dict[str, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        # we know the landmarks so no estimation required!
        return self.landmarks

    def init(self, init_state: Pose) -> None:
        self.particles = LocalizationParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.4),
        )

    # We incrementally implement methods below using the `implementation_for` helper
    def motion_model(self, command: Command):
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command, measurements: Sequence[Measurement]):
        raise NotImplementedError()


def plot_particles(samples: LocalizationParticles, fov=((-6, -6), (20, 20))):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
        particles: a set of particles for localization. This function only plots
            the first two components: x and y.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples.x, samples.y, 'g+')
    plt.show()

### 2.1 Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that updates the set of sampled poses (particles)
drawn from prior pose random variable $\mathbf{X}_t$
using a control $[\Delta p_t, \Delta \theta_t]$ to a set of samples drawn from the posterior $X_{t+1}$.

**Note**
In this project, we will use a helper decorator `implementation_for` to implement
class methods outside of the class.
The decorator `@implementation_for(Localization)` simply inserts the declared
function as a method of the Localization class.

**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def localization_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = Localization(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        landmarks=sim.landmarks,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"
localization_motion_model_problem_test()

print('Tests passed.')


### Experiment 3

Now that you have implemented `motion_model`, let's play with the noise
parameters. Let's first use the motion parameters of `motion_noise_covariance=np.eye(2)`:
- Initialize your particle filter using the default parameters (but pass in the above motion noise parameter and use 1000 particles), 
and step through the motion model twice. 
- Use a Command of $\Delta_p = 1m$ and $\Delta_\theta = 0$ for both times.
- Please generate plots of the $[x, y]$ component of the particles after the first and
second motions (two separate plots).

This is a pretty noisy motion! You should see the particles much
more spread out than they were at the start. Why is the distribution long and
skinny after one motion and then really spread out after the second motion?


<b>Submission Material 3:</b> In your submitted pdf, please include the generated 
figures. Name them Figure 3a and 3b.

### Experiment 4

Now let's try reducing the noise parameters. Please perform experiment 3 again, but set 
`motion_noise_covariance=np.diag([1e-2, np.deg2rad(.5)**2])`.
Please generate two plots same as in experiment 3. 
The particle set should be much more compact than before and track the motion you commanded.

<b>Submission Material 4:</b> In your submitted pdf, please include the generated figures.
Name them Figure 4a and 4b.

### Experiment 5

For the two experiments above, we're taking pretty big motions,
commanding a meter at a time. A mobile robot would generally send commands and
get measurements at a much higher frequency. 
For example, the robot sends commands at 10 Hz and travels at 1 m/s. 
Please perform an experiment that moves the robot the same 2m forward but in .1 m increments. 
Please upload a plot of the particles' $[x, y]$ component after 2 m.

In this question, please use the same reduced noise parameters: `motion_noise_covariance=np.diag([1e-2, np.deg2rad(.5)**2])` as in the previous experiment.

<b>Submission Material 5:</b> In your submitted pdf, please include the generated figure.
Name this Figure 5.

### Experiment 6

Even though all we did is changed how often we sent a command to the robot,
the sample set is much more spread out. You might think that the uncertainty
in the robot's position is independent of things like control frequency, but
this plot should show you that they aren't. When we adjust the control
frequency, we also have to adjust the model parameters.

Let's set `motion_noise_covariance=np.diag([1e-3, np.deg2rad(.5)**2])`. 
In your submitted pdf, please include two plots of the $[x, y]$ component of the 1000 particles after 2m. 

The particle set should be much more compact again, and track the motion you commanded.

<b>Submission Material 6:</b> In your submitted pdf, please include the generated figure.
Name this Figure 6.

### Experiment 7

What happens when our motion model has worse rotation noise than translation
noise? 
Let's set `motion_noise_covariance=np.diag([1e-3, np.deg2rad(30)**2])`.
Please generate plots of the $[x, y]$ component of the samples
after 1m, 2m and 3m of motion made in 1m increments. You should see the samples are
tightly contained, but are in a curved shape. This shape is a result of the
non-linear relationship between orientation and position, and is one of the
reasons we often use sampling for localization, rather than a parametric form
such as a Gaussian.

<b>Submission Material 7:</b> In your submitted pdf, please include the generated figures.
Name them Figure 7a, 7b, and 7c.


### 2.2 Localization: Computing the Importance Weights

Let us now add in a measurement model.
Let us assume that we have a set of landmarks given by `grid_of_landmarks()` (i.e., the default set of landmarks of a Simulator).
The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = \texttt{atan2}(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how the values are simulated.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that nature applied. (That would be omniscience!)
What we can do is work out how likely each measurement is. More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. 

In particular, to compute the importance weight of a particle:
- Compute the _predicted measurements_ from the current state of the robot's pose and the landmark locations.
- Compute the errors $\mathit{err}$ between the predicted and true measurements.
- Compute the likelihood of the $\mathit{err}$ as the unnormalized importance weights.
The likelihood of the error is the evaluation of the probability density function (PDF) of the two-dimensional Gaussian distribution with covariance `sensor_noise_covariance` at $\mathit{err}$.
- Finally, normalize the weights so that they sum to one.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.

There are a few technicalities during this process:
- We highly recommend you normalize **all the rotational values** (in radians) to the interval $[-\pi, \pi]$ (see function `normalize_angles`).
Notably, the rotational component of the $\mathit{err}$, which we denote as $\mathit{err}_\theta$, is a rotational
value in radians. Therefore, theoretically, the likelihood of $\mathit{err}_\theta$ should have been
the evaluation of the PDF of a [wrapped Gaussian distribution](https://en.wikipedia.org/wiki/Wrapped_normal_distribution).
In other words, to evaluate the likelihood of
$\mathit{err}_\theta$, we should have taken the sum of the Gaussian PDFs at the points
$\mathit{err}_\theta + 2k\pi$ for all $k \in \mathbb{Z}$.
Nonetheless, since our sensor noise is relatively small, we expect only the single point at $k=0$
$\mathit{err}_\theta \in [-\pi, \pi]$ to have a relatively large value, and all other
points to have close-to-zero values. Therefore, we may use the Gaussian distribution as a
good approximation to the true wrapped Gaussian distribution.
- The likelihood for the particles can become really small. For numerical stability,
you will want to compute the _log-likelihood_ instead of the likelihood.
To that end, you may:
    - Use the utility `multivariate_normal_logpdf` that we provided you to evaluate the log PDF.
    - Take the sum (instead of multiplications) of the log-likelihoods to compute the unnormalized log-weight of each particle.
    - Normalize the log-weights. You should use [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html) to evaluate
the log-sum of the weights. In short, The `logsumexp` function calculates the following expression in a more
numerically stable way:
$$
\texttt{logsumexp}(x_1, ... x_n) = \log \left( \sum_{i=1}^{n} \exp(x_i) \right).
$$ See <a href="https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/" target="_blank">this blog</a> for a more detailed description of this function.
    - Use `numpy.exp` to undo the logarithm from the log-weights and return the weights.


For reference, our solution is **29** line(s) of code.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def localization_compute_weights_test(_, infer: Localization,
                                      measurements: Sequence[Measurement],
                                      result_weights: np.ndarray):
    weights = infer.compute_weights(measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

localization_compute_weights_test(compute_weights, Localization(motion_noise_covariance=np.array([[0.001     , 0.        ],  [0.        , 0.00274156]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=20, particles=LocalizationParticles(x=np.zeros((20,), dtype=np.float64), y=np.zeros((20,), dtype=np.float64), theta=np.zeros((20,), dtype=np.float64))), [Measurement(landmark_id='landmark-35', r=22.753147219062917, b=0.7738698392857971), Measurement(landmark_id='landmark-25', r=12.640422650443282, b=0.009154262150241616), Measurement(landmark_id='landmark-15', r=8.408602536838048, b=1.1387038391295914), Measurement(landmark_id='landmark-3', r=10.248271955129297, b=2.117092341243524), Measurement(landmark_id='landmark-32', r=15.78868726666365, b=0.1345498076509456), Measurement(landmark_id='landmark-12', r=5.033579786955029, b=-0.7817917913663452), Measurement(landmark_id='landmark-7', r=-2.3250307746388343, b=-0.01909317455769924), Measurement(landmark_id='landmark-20', r=7.698360962746094, b=0.399745227278584), Measurement(landmark_id='landmark-5', r=15.948163519613333, b=1.7881725941556874), Measurement(landmark_id='landmark-2', r=6.068484785866513, b=2.4471709441565186), Measurement(landmark_id='landmark-26', r=12.520575977729484, b=0.44099698774324514), Measurement(landmark_id='landmark-24', r=11.983915967186904, b=-0.291075514011446), Measurement(landmark_id='landmark-8', r=4.903470181651809, b=1.5790004474613593), Measurement(landmark_id='landmark-30', r=15.748923253116834, b=-0.32541437620132907), Measurement(landmark_id='landmark-11', r=15.542274174332661, b=1.5900119763018357), Measurement(landmark_id='landmark-4', r=11.639492457134782, b=1.8742928687102223), Measurement(landmark_id='landmark-34', r=19.84077499008552, b=0.6906987897814902), Measurement(landmark_id='landmark-9', r=8.21465912250634, b=1.6018084460172888), Measurement(landmark_id='landmark-27', r=13.76837649243762, b=0.5766916802305041), Measurement(landmark_id='landmark-14', r=6.44082971955371, b=0.9157246165771075), Measurement(landmark_id='landmark-23', r=16.629478287894198, b=1.2392634902611783), Measurement(landmark_id='landmark-21', r=12.659583922767066, b=0.8535804455254041), Measurement(landmark_id='landmark-13', r=4.264455630329303, b=-0.027394933553940907), Measurement(landmark_id='landmark-1', r=5.458020683536959, b=3.312657462648057), Measurement(landmark_id='landmark-6', r=5.801634869866125, b=-1.4560318732949584), Measurement(landmark_id='landmark-33', r=18.245924230657273, b=0.3582019162682134), Measurement(landmark_id='landmark-18', r=8.939817776879076, b=-0.40635936363131575), Measurement(landmark_id='landmark-0', r=4.368492785742826, b=-2.3217135857051487), Measurement(landmark_id='landmark-31', r=16.42986369482223, b=0.06074118633851044), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-19', r=7.563564752856779, b=-0.10208447443373762), Measurement(landmark_id='landmark-10', r=13.739367877130134, b=1.5275199517585014), Measurement(landmark_id='landmark-16', r=12.97808027013372, b=1.2264810610293408), Measurement(landmark_id='landmark-28', r=18.554035627279266, b=0.9006213960947502), Measurement(landmark_id='landmark-29', r=20.633352622824916, b=0.7350027054140555), Measurement(landmark_id='landmark-17', r=16.54445147673053, b=1.3854805390653757)], np.full((20,), 0.05000000000000003, dtype=np.float64))
print('Tests passed.')

### 2.3 Localization: Particle Filter Update

What we want you to do is to write a method `update` that takes in a
set of samples, the most recent command, and the most recent measurements, and then:
  * Use the motion_model method to update the sample based on the command
  * Use compute_weights to compute the sample weights from the
measurement,
  * Resample a new set of samples,
  * Blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag([1e-3, 1e-3, np.deg2rad(1)**2])`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **12** line(s) of code.

In addition to all the utilities defined at the top of this notebook, the following functions are available: `compute_weights`, `motion_model`. 

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def localization_update_test(_, infer: Localization, command: Command,
                             measurements: Sequence[Measurement],
                             pose: Pose):
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

localization_update_test(update, Localization(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 9.27917724e-08]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=100, particles=LocalizationParticles(x=np.zeros((100,), dtype=np.float64), y=np.zeros((100,), dtype=np.float64), theta=np.zeros((100,), dtype=np.float64))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='landmark-4', r=12.649236370894611, b=1.8925066397489418), Measurement(landmark_id='landmark-16', r=12.649751063323961, b=1.249077726801277), Measurement(landmark_id='landmark-20', r=8.943736240625999, b=0.46375775715344025), Measurement(landmark_id='landmark-35', r=22.628720998014654, b=0.7856866607567657), Measurement(landmark_id='landmark-34', r=19.999296264764194, b=0.6431156393698316), Measurement(landmark_id='landmark-10', r=11.999376725537463, b=1.5708089154080955), Measurement(landmark_id='landmark-12', r=5.654529218717742, b=-0.7854648111495861), Measurement(landmark_id='landmark-7', r=-0.0012459109472530653, b=-0.00022306139218380394), Measurement(landmark_id='landmark-33', r=17.887999561015462, b=0.4635512584632948), Measurement(landmark_id='landmark-17', r=16.492834133007015, b=1.3261352314007253), Measurement(landmark_id='landmark-8', r=3.999871465337056, b=1.5712125753715283), Measurement(landmark_id='landmark-31', r=15.999334805326514, b=0.00010707609058078074), Measurement(landmark_id='landmark-1', r=4.000903470181652, b=3.141621291373365), Measurement(landmark_id='landmark-29', r=19.999256500750647, b=0.9270144443957444), Measurement(landmark_id='landmark-15', r=8.943814184173492, b=1.1072157930644515), Measurement(landmark_id='landmark-5', r=16.491412884287104, b=1.815711271397861), Measurement(landmark_id='landmark-18', r=8.944112684989244, b=-0.46348285801429606), Measurement(landmark_id='landmark-25', r=12.000214659122506, b=0.00010825271769035358), Measurement(landmark_id='landmark-23', r=17.8878899913889, b=1.1071092352234257), Measurement(landmark_id='landmark-21', r=11.314492474454822, b=0.7858530885395344), Measurement(landmark_id='landmark-11', r=15.998740934467897, b=1.5712574943489113), Measurement(landmark_id='landmark-32', r=16.493768377894426, b=0.24521666418979546), Measurement(landmark_id='landmark-28', r=16.970827204107472, b=0.7853025370396719), Measurement(landmark_id='landmark-30', r=16.49388052315418, b=-0.24438153429639173), Measurement(landmark_id='landmark-27', r=14.424006736725824, b=0.5884032070631326), Measurement(landmark_id='landmark-3', r=8.944629290409818, b=2.0340758608916567), Measurement(landmark_id='landmark-22', r=14.422200647722839, b=0.9829936969482067), Measurement(landmark_id='landmark-0', r=5.655565888028631, b=-2.3560741291298712), Measurement(landmark_id='landmark-14', r=5.6572841131872025, b=0.785610190136083), Measurement(landmark_id='landmark-24', r=12.647926522706761, b=-0.32195212052680294), Measurement(landmark_id='landmark-2', r=5.656417814245238, b=2.3558381481535373), Measurement(landmark_id='landmark-6', r=4.00173936787713, b=-1.5709473898414392), Measurement(landmark_id='landmark-9', r=8.00032896962946, b=1.5707175610932667), Measurement(landmark_id='landmark-19', r=8.001583472878803, b=0.00040220495707190147), Measurement(landmark_id='landmark-26', r=12.649743993296344, b=0.3210793269023251), Measurement(landmark_id='landmark-13', r=4.00005202897426, b=0.00020826272337815132)], Pose(x=0.9858299248258179, y=-0.00020686081759629056, theta=0.7682645354557489))
print('Tests passed.')

#### Experiment 9

Let's visualize our particle filter.

Please generate a trajectory plot and an animation of the robot carrying out the 
localization inference. 
In particular:
- Use the default Simulator parameters (i.e. simply use `Simulator()`)
- Run localization with `num_particles=100`.
- Plot (and also animate) the obtained results.

If everything is working, your estimated and ground truth trajectories should 
match up closely, but not exactly. 
(You should be able to inspect the estimated and ground truth trajectories manually, and see that they differ by some small errors at each step.)


Here are some handy functions for your experiments:
- `run_inference`: It takes in a simulator instance and runs
inference in a particular mode --- for this question, you should pass in 
the "localization" mode. It returns a `SimulationResult` instance. 
- `plot_simulation_result`: It visualizes a `SimulationResult`.
- `render_animation`: It creates an animated visualization of a `SimulationResult`.

<b>Submission Material 9:</b> 
In your submitted pdf, please include the generated figure.
Name this Figure 9.
Additionally include the generated animation. Name this 'localization.mp4'.

# 3. Particle Filter SLAM 

Now, what if we don't know the landmark positions in advance?  
If the robot knew its locations exactly, it could use its observations of the landmarks to build a map of the 
landmarks' locations in absolute coordinates. But the robot is as lost as it was before --- what can we do?
This problem is known as *simultaneous localization and mapping* (SLAM) --- we need to estimate both the robot's pose *and* the map (i.e., the positions of the landmarks).

In this section, we will use the same algorithm from last section, the particle filter, to approach the SLAM problem.
Recall each particle in the localization problem only contains a robot's pose.
In the SLAM case, each particle will now contain a complete "possible world" --- made up of a robot pose and a location of each landmark that is known by to the robot so far.

### Utilities

In [ ]:


@dataclasses.dataclass(frozen=True)
class SLAMParticles:
    """A batch of samples for SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape (num_particles, num_landmarks, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape, "shape must match"
        assert self.x.shape[0] == self.landmarks_loc.shape[
            0], "shape must match"
        assert self.landmarks_loc.shape[1] == len(
            self.landmarks_id_to_idx), "shape must match"
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass
class SLAM(Inference):

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: SLAMParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = SLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the l
        andmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt
        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

### 3.1) SLAM: Update by Motion Model

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$
and updates the particles by drawing from the posterior $X_{t+1}$ using the motion model.

This function behaves the same as `Localization.motion_model` for the pose
components of the particles, and simply keeps the landmarks components
(`SLAMParticles.landmarks_id_to_idx` and `SLAMParticles.landmarks_loc`)
of the particles the same.

**Hint**

Please take a look at the fields of the `SLAMParticles` class.
Note that `landmarks_id_to_idx` stores all the landmarks the robot has encountered so far.
It is a mapping from a landmark identifier (a string) to an index (an integer).
You can use the integer index to index into the `landmarks_loc` array.
In particular, `landmarks_loc` is an array of shape `(num_particles, len(landmarks_id_to_idx), 2)`.
Therefore, to access the coordinate of the landmark with the identifier `"Stata"` contained
in the $i$-th particle:
```python
# assert isinstance(particles, SLAMParticles)
loc = particles.landmarks_loc[i, particles.landmarks_id_to_idx["Stata"]]
```


For reference, our solution is **11** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def motion_model(self: SLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def slam_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)),
                    landmarks=[])
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = SLAM(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

slam_motion_model_problem_test()

print('Tests passed.')

### 3.2 SLAM: Tracking New Landmarks

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we have to match newly observed landmarks to ones we have seen before,
and add landmarks that we are seeing for the first time to our map.  
In some situations, the matching is difficult, but for our purposes,
we’ll assume that the landmarks are uniquely identifiable, so we know exactly
which ones we are observing on each step.

One way to handle missing data due to sensing range is to explicitly model the
sensing range in our probabilistic model of the sensors.
However, this approach would induce a difficult inference problem.  
Instead, we will take the approach of treating the missingness as uninformative.
In particular, we will consider sensor data within-range as observed variable,
and sensor data out-of-range as unobserved variables.
When we see a new landmark for the first time, we will assume that our belief
about the location of that landmark has a mean equal to that first observed location
and a standard variance (new_landmarks_init_covariance).
Note, though, that in practice it might be more common to wait until we have gotten
a few observations and use their mean as the mean of the distribution for
initializing our particles.

Please implement a function that takes a set of measurements and updates the
particles to include potentially new landmarks that are not yet tracked by the
particles.

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked)
landmarks.
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add a Gaussian noise of covariance `self.new_landmarks_init_covariance` to this predicted location.
    - This noisy location becomes the landmark's location in the current particle.
- Update `self.particles.landmarks_id_to_idx` to maintain the invariance that it
holds a mapping from each tracked unique landmark identifier to the index of
that landmark in `self.particles.landmarks_loc`.

For reference, our solution is **24** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def add_new_landmarks(self: SLAM, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added
        before).
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def slam_add_new_landmarks_test(add_new_landmarks, infer: SLAM,
                                measurements: Sequence[Measurement],
                                results: SLAMParticles):
    add_new_landmarks(infer, measurements)
    assert (np.allclose(infer.particles.x, results.x) and np.allclose(
        infer.particles.y,
        results.y)), "pose component of the particles should not change"
    assert infer.particles.landmarks_id_to_idx == results.landmarks_id_to_idx, "indexing mapping must be the same"
    assert np.allclose(
        infer.particles.landmarks_loc, results.landmarks_loc
    ), "landmark locations should match exactly, since we applied zero initialization noise"

slam_add_new_landmarks_test(add_new_landmarks, SLAM(motion_noise_covariance=np.zeros((2, 2), dtype=np.float64), sensor_noise_covariance=np.zeros((2, 2), dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=3, particles=SLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1}, landmarks_loc=np.array([[[ 2,  3],   [ 1,  1]],   [[ 1,  2],   [ 0,  0]],   [[ 0,  1],   [-1, -1]]], dtype=np.int64))), [Measurement(landmark_id='Stata', r=100, b=0.1), Measurement(landmark_id='Cafe-Luna', r=1, b=1.5707963267948966)], SLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1, 'Cafe-Luna': 2}, landmarks_loc=np.array([[[ 2.000000e+00,  3.000000e+00],   [ 1.000000e+00,  1.000000e+00],   [ 6.123234e-17,  1.000000e+00]],   [[ 1.000000e+00,  2.000000e+00],   [ 0.000000e+00,  0.000000e+00],   [ 1.000000e+00,  2.000000e+00]],   [[ 0.000000e+00,  1.000000e+00],   [-1.000000e+00, -1.000000e+00],   [ 2.000000e+00,  3.000000e+00]]], dtype=np.float64)))
print('Tests passed.')

### 3.3 SLAM: Computing the Importance Weight

Please implement the `SLAM.compute_weights` function that takes a
sequence of measurements and returns the importance weights of the particles.

This function is similar to `Localization.compute_weights`, except that it takes
into account the landmarks location tracked in the particles.


For reference, our solution is **17** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def compute_weights(self: SLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def slam_compute_weights_test(_, infer: SLAM,
                              measurements: Sequence[Measurement],
                              result_weights: np.ndarray):
    weights = infer.compute_weights(measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

slam_compute_weights_test(compute_weights, SLAM(motion_noise_covariance=np.array([[0.001     , 0.        ],  [0.        , 0.00274156]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), new_landmarks_init_covariance=np.array([[10.,  0.],  [ 0., 10.]], dtype=np.float64), num_particles=20, particles=SLAMParticles(x=np.zeros((20,), dtype=np.float64), y=np.zeros((20,), dtype=np.float64), theta=np.zeros((20,), dtype=np.float64), landmarks_id_to_idx={'landmark-17': 0, 'landmark-24': 1, 'landmark-9': 2, 'landmark-16': 3, 'landmark-33': 4, 'landmark-4': 5, 'landmark-1': 6, 'landmark-34': 7, 'landmark-26': 8, 'landmark-20': 9, 'landmark-2': 10, 'landmark-31': 11, 'landmark-19': 12, 'landmark-21': 13, 'landmark-28': 14, 'landmark-27': 15, 'landmark-30': 16, 'landmark-14': 17, 'landmark-13': 18, 'landmark-25': 19, 'landmark-32': 20, 'landmark-7': 21, 'landmark-5': 22, 'landmark-12': 23, 'landmark-11': 24, 'landmark-10': 25, 'landmark-29': 26, 'landmark-8': 27, 'landmark-3': 28, 'landmark-22': 29, 'landmark-0': 30, 'landmark-6': 31, 'landmark-23': 32, 'landmark-18': 33, 'landmark-15': 34, 'landmark-35': 35}, landmarks_loc=np.array([[[ 2.92582568e+00,  2.16864042e+01],   [ 1.14022409e+01, -4.59991183e+00],   [ 2.19765309e+00,  8.50585103e+00],   [ 2.84512281e+00,  1.11639161e+01],   [ 1.70773609e+01,  9.23047418e+00],   [-2.32899042e+00,  8.93408872e+00],   [ 3.81750072e+00, -4.54250241e+00],   [ 1.06863265e+01,  1.33089427e+01],   [ 1.22991120e+01,  1.77182852e+00],   [ 4.47945276e+00,  1.22660540e+01],   [-3.97612185e+00,  5.10560066e+00],   [ 1.63328321e+01,  1.84530550e+00],   [ 1.06110992e+01,  2.38917241e+00],   [ 6.86409326e+00,  5.92729795e+00],   [ 5.94423118e+00,  9.43099878e+00],   [ 1.21512233e+01,  1.05396767e+01],   [ 1.79091114e+01, -4.28674005e+00],   [ 2.65520331e-02, -4.50425512e+00],   [ 6.98805996e+00, -1.90115314e+00],   [ 1.12573904e+01,  2.07745118e+00],   [ 9.71827213e+00,  4.50311350e+00],   [ 7.74152841e+00,  1.68986760e+00],   [-2.48127772e+00,  1.13574959e+01],   [ 1.29088684e+01, -8.60038533e+00],   [ 4.04041145e+00,  1.09961375e+01],   [ 5.74477492e+00,  1.74054813e+01],   [ 8.45472848e+00,  1.79934406e+01],   [ 1.63780436e+00,  1.36582604e-01],   [-9.27599216e+00,  3.94754605e+00],   [ 9.67743597e+00,  1.03034447e+01],   [-5.83283229e+00, -2.38807230e-01],   [ 1.35835028e+00, -8.12297715e+00],   [ 4.71415654e+00,  1.77609684e+01],   [ 8.19516394e+00, -3.28064072e+00],   [ 1.08261317e+01,  9.17852836e+00],   [ 1.23252946e+01,  1.74725445e+01]],   [[ 1.20355363e+00,  2.11610348e+01],   [ 1.08696325e+01, -3.43148440e+00],   [ 7.37637074e-01,  7.96420892e+00],   [-2.88895463e+00,  8.98217369e+00],   [ 1.46934326e+01,  6.95469487e+00],   [-4.27971867e+00,  8.36815501e+00],   [-5.93875603e+00, -1.23785792e+00],   [ 1.42097666e+01,  1.18931254e+01],   [ 1.05530322e+01,  5.41542446e+00],   [ 6.07299074e+00,  4.11763125e+00],   [-5.44701499e+00,  9.42246647e-01],   [ 1.73043899e+01, -2.10249444e+00],   [ 7.25359525e+00, -1.58180299e-01],   [ 3.94797597e+00,  5.82201632e+00],   [ 9.20048050e+00,  1.06710866e+01],   [ 1.11043803e+01,  4.70547362e+00],   [ 1.53188413e+01,  9.28997701e-01],   [ 3.93947331e+00,  7.90016265e+00],   [ 4.44166898e+00,  2.41026332e+00],   [ 1.02265069e+01,  1.99312335e+00],   [ 1.45865131e+01,  3.67192918e+00],   [-1.25926336e+00, -4.62093917e+00],   [ 1.11388565e+00,  1.32867749e+01],   [ 1.00711010e+01, -5.57945600e+00],   [-4.76595647e+00,  1.96531103e+01],   [-6.95384989e-01,  1.40863694e+01],   [ 1.00092958e+01,  1.81763598e+01],   [-3.15515438e+00,  5.33898960e+00],   [-3.15037480e+00,  5.20634333e+00],   [ 1.63243143e+01,  1.22806228e+01],   [ 3.19473711e+00,  1.76449506e+00],   [-8.45449522e-01, -7.03771200e+00],   [ 7.23683931e+00,  1.03192111e+01],   [ 7.69472796e+00, -2.81401135e+00],   [ 1.06669381e+01,  5.14630772e+00],   [ 1.78413302e+01,  1.30034413e+01]],   [[ 2.36197476e+00,  1.46272545e+01],   [ 1.38212703e+01, -2.64079198e+00],   [ 2.80769303e+00,  9.89821522e+00],   [ 4.07348639e+00,  1.47906010e+01],   [ 2.11472649e+01,  4.32887227e+00],   [-1.39411649e+00,  5.67706794e+00],   [-3.73910843e-01, -1.49619070e+00],   [ 1.56993487e+01,  1.35604356e+01],   [ 1.21055912e+01,  6.30870429e+00],   [ 1.26138920e+01,  6.15746363e+00],   [-6.43238697e-01,  2.38363174e+00],   [ 1.53016293e+01, -1.18234570e+00],   [ 1.22013553e+01,  1.38251616e+00],   [ 6.44663447e+00,  7.33865668e+00],   [ 1.47435572e+01,  1.17743617e+01],   [ 8.29310028e+00,  7.65216420e+00],   [ 9.75168373e+00, -7.78310024e-01],   [ 2.68870070e+00,  2.79443480e+00],   [ 6.11515023e+00, -4.35487155e+00],   [ 1.88328278e+01,  1.42719085e-01],   [ 1.25083785e+01,  8.83861806e+00],   [ 3.58383950e+00,  7.84316276e+00],   [-4.28521527e+00,  1.59810236e+01],   [ 1.13596833e+00, -5.15131239e+00],   [-4.31987925e+00,  1.62654060e+01],   [-4.21498679e+00,  1.75400466e+01],   [ 6.58204548e+00,  7.79578082e+00],   [-3.85160922e+00,  8.00518390e-01],   [-8.32868904e+00,  1.15876749e+01],   [ 7.53073201e+00,  1.06528802e+01],   [-2.11812750e+00, -2.39154509e+00],   [ 2.84619999e+00, -9.24247033e+00],   [ 6.78128501e+00,  1.82122425e+01],   [ 1.88018686e+00, -7.94306920e+00],   [ 7.42571319e+00,  6.07134619e+00],   [ 1.80122543e+01,  1.95233420e+01]],   [[ 4.27379130e-01,  1.73604037e+01],   [ 1.64456027e+01, -3.62070354e+00],   [-3.32651412e+00,  1.02414899e+01],   [ 5.31390976e+00,  1.52720447e+01],   [ 1.83878873e+01, -3.52523463e+00],   [-1.05825337e+00,  1.71127359e+01],   [-2.93891836e-01,  6.41441137e-01],   [ 1.78935504e+01,  1.13409569e+01],   [ 1.36460551e+01,  3.51414721e+00],   [ 5.52890479e+00,  1.75109319e+00],   [-7.50295510e+00, -9.25633557e-01],   [ 1.19510945e+01,  6.03993453e+00],   [ 1.12878304e+01, -1.90346583e+00],   [ 6.81612058e+00,  7.20548905e+00],   [ 9.37681331e+00,  1.21120450e+01],   [ 1.37528895e+01,  7.12199415e+00],   [ 1.70773571e+01, -4.06043868e-01],   [ 3.16049934e+00,  5.17911261e+00],   [ 2.36761949e+00, -1.28063689e-01],   [ 1.16486740e+01, -1.10555580e-02],   [ 1.47123684e+01,  7.40079547e+00],   [ 1.80540355e+00,  1.66880689e+00],   [-5.95086849e+00,  1.25552379e+01],   [ 7.13278356e+00, -1.92584082e+00],   [-1.88085066e+00,  2.19473953e+01],   [ 4.17101482e+00,  2.07654537e+01],   [ 1.04267267e+01,  1.34984869e+01],   [-2.43399926e+00,  9.21765638e-01],   [-3.18813650e+00,  1.29012215e+00],   [ 1.21362422e+01,  9.81588160e+00],   [-4.82878549e+00, -6.70917890e+00],   [-6.09514424e-01, -5.69212642e+00],   [ 8.12706555e+00,  1.71784690e+01],   [ 8.03057158e+00, -6.42717220e+00],   [ 7.57540244e+00,  7.90204087e+00],   [ 1.56755635e+01,  2.03134274e+01]],   [[ 9.82448950e-01,  1.33699333e+01],   [ 1.66211781e+01, -8.77622097e+00],   [-4.37622355e+00,  1.00469209e+01],   [ 4.05777612e+00,  1.38889319e+01],   [ 1.58340372e+01,  6.04343938e+00],   [-4.13530662e+00,  1.56812073e+01],   [-2.93371886e+00, -4.78329392e-01],   [ 2.13657633e+01,  1.03583874e+01],   [ 1.86431973e+01,  3.17752054e+00],   [ 1.22775843e+01, -3.07617005e-01],   [-3.86052925e+00, -1.59423712e+00],   [ 1.79979902e+01, -3.36510279e+00],   [ 9.80052867e+00, -8.19337392e-01],   [ 4.38809061e+00,  1.72868499e+00],   [ 9.83377687e+00,  1.74028916e+01],   [ 1.03658081e+01,  1.01355318e+01],   [ 1.58343967e+01, -4.36922768e-01],   [-1.75614552e+00,  3.00353229e+00],   [ 6.34824122e+00, -5.54104959e-01],   [ 1.24021656e+01,  3.00137678e-01],   [ 1.77977000e+01,  6.55004487e+00],   [ 1.48195893e+00,  2.12521080e+00],   [-1.75719275e+00,  1.70340421e+01],   [ 1.07061193e+01, -4.29175870e+00],   [-1.15914369e+00,  1.56513374e+01],   [ 1.77532522e+00,  1.73096873e+01],   [ 9.02757988e+00,  1.30723889e+01],   [ 3.05854953e+00,  2.23958684e+00],   [-4.38988796e+00,  7.97187337e+00],   [ 8.21682262e+00,  1.32622180e+01],   [-6.70189942e+00, -6.49550670e+00],   [ 2.03913268e-02, -1.08140026e+01],   [ 4.17071274e+00,  2.20092399e+01],   [ 1.22904770e+01, -3.62615539e+00],   [ 9.75635832e+00,  4.29936329e+00],   [ 1.40539976e+01,  1.89360226e+01]],   [[ 7.03263264e+00,  1.76964266e+01],   [ 4.60939773e+00, -7.14827651e+00],   [ 1.27312300e+00,  6.33499727e+00],   [ 1.13374387e+01,  1.64861933e+01],   [ 1.78886996e+01,  2.41245809e+00],   [-3.95564063e+00,  1.21576750e+01],   [ 1.89170105e+00,  2.81835972e+00],   [ 1.89578166e+01,  7.34972775e+00],   [ 1.60505575e+01, -8.55809831e-01],   [ 6.25872778e+00,  6.28925840e+00],   [-4.82039453e+00,  2.34940348e-01],   [ 1.27296225e+01, -4.42568805e+00],   [ 1.05854221e+01,  4.05931302e+00],   [ 7.73317854e+00,  5.58353677e+00],   [ 6.79393480e+00,  1.47863360e+01],   [ 1.82866476e+01,  1.55096827e+00],   [ 1.26519118e+01, -5.35843816e+00],   [ 2.23652559e+00,  4.99745668e+00],   [ 8.45066262e+00,  3.15161719e+00],   [ 1.43439707e+01, -6.27885803e-01],   [ 1.06224679e+01,  3.08959712e+00],   [ 6.23816533e+00, -1.03555688e-01],   [-5.03034361e+00,  1.57806962e+01],   [ 8.42164798e+00, -8.25270797e+00],   [ 2.43571170e+00,  1.81793533e+01],   [ 6.16479459e-01,  1.36891468e+01],   [ 1.17523712e+01,  1.55686794e+01],   [-8.92622536e-01, -3.75579037e-02],   [-2.61779010e+00,  6.43222202e+00],   [ 4.94266311e+00,  1.10971104e+01],   [ 1.54758048e-01, -3.44887524e+00],   [-2.17128653e+00, -4.45676288e+00],   [ 1.46371118e+01,  1.15149411e+01],   [ 1.11037831e+01, -2.94581318e+00],   [ 9.51452904e+00,  1.07235569e+01],   [ 1.44615962e+01,  1.16375779e+01]],   [[ 6.76666007e+00,  1.34333299e+01],   [ 9.65164439e+00, -4.55528670e-01],   [-3.11586057e+00,  1.18967993e+01],   [ 4.36655524e+00,  1.45652997e+01],   [ 1.70955463e+01,  7.63412287e-01],   [-9.46145658e+00,  1.20765597e+01],   [ 1.21533053e-01,  2.93217558e-01],   [ 1.30932977e+01,  1.03312310e+01],   [ 1.13255270e+01,  4.82695827e-01],   [ 1.11598999e+01, -5.44572868e-01],   [-2.48462873e+00,  4.63164555e+00],   [ 1.85205804e+01, -2.70875327e+00],   [ 9.22384810e+00,  6.93887074e+00],   [ 1.01443134e+01,  7.16761659e+00],   [ 1.31548064e+01,  1.21710531e+01],   [ 1.82221172e+01,  1.03455534e+01],   [ 1.57440438e+01, -2.41143421e+00],   [ 2.78524374e+00,  8.10440769e+00],   [-1.10053636e+00, -5.11247628e+00],   [ 1.72884223e+01,  8.47014038e+00],   [ 1.56439534e+01,  5.96573055e+00],   [ 3.42121120e+00, -1.95446914e+00],   [-3.07580367e+00,  1.64848229e+01],   [ 7.79534634e+00, -9.02793672e+00],   [-3.64189790e+00,  2.03566426e+01],   [-8.35796080e-01,  9.14745997e+00],   [ 1.75193382e+01,  1.67776638e+01],   [-1.92418112e+00, -2.64171851e+00],   [-2.38708425e+00,  1.03253968e+01],   [ 7.02185695e+00,  7.08704595e+00],   [-4.12598783e+00, -5.93920508e+00],   [-3.22746302e+00, -6.06038162e+00],   [ 5.18790090e+00,  1.68862420e+01],   [ 1.18177929e+01, -6.24950497e+00],   [ 1.51466574e+00,  1.25272266e+01],   [ 1.20786267e+01,  1.48775671e+01]],   [[ 4.88970091e+00,  1.79698135e+01],   [ 1.18349553e+01, -2.62977245e+00],   [-1.50352834e+00,  4.38950342e+00],   [ 7.81493092e+00,  2.14253020e+01],   [ 2.14870163e+01,  6.39299468e+00],   [ 1.95380914e+00,  1.25197544e+01],   [-3.18397564e+00,  1.50680025e+00],   [ 1.53927226e+01,  7.16250141e+00],   [ 8.74419823e+00,  6.04580579e+00],   [ 2.11419159e+00,  6.83748859e+00],   [-9.49959152e+00, -3.29144066e+00],   [ 9.51264765e+00,  6.68977578e+00],   [ 8.28262426e+00,  3.46844381e-01],   [ 1.25387292e+01,  9.80368663e+00],   [ 7.04950268e+00,  1.46465773e+01],   [ 1.18763216e+01,  6.15454159e+00],   [ 1.60915967e+01,  8.37744703e-01],   [ 1.21938350e+00,  5.54623749e+00],   [ 3.31541848e+00, -5.72098100e+00],   [ 1.01386140e+01,  2.27237267e+00],   [ 1.82655992e+01,  8.77570858e+00],   [ 3.23130003e+00, -2.48643424e+00],   [ 2.07861546e+00,  1.70424329e+01],   [ 2.63156900e+00, -4.67950911e+00],   [ 7.95673197e-01,  1.77086205e+01],   [ 4.10513157e+00,  1.57789312e+01],   [ 1.23283283e+01,  1.39448303e+01],   [ 3.34864032e-01,  2.85285370e+00],   [ 1.18628947e+00,  2.97165263e+00],   [ 7.54027174e+00,  8.72553789e+00],   [-2.98865872e+00, -1.10042514e+01],   [ 3.27739951e+00, -5.69051771e+00],   [ 2.67923199e+00,  1.49643295e+01],   [ 1.18208598e+01, -8.62695410e+00],   [ 5.85162618e+00,  4.24600169e+00],   [ 1.32938433e+01,  1.70490434e+01]],   [[ 4.79620212e+00,  1.45858226e+01],   [ 1.35081584e+01, -1.50025974e+00],   [-1.11750315e+00,  1.19876295e+01],   [ 7.92844819e+00,  1.22004837e+01],   [ 1.41068608e+01,  6.81985663e-01],   [-4.24426991e+00,  1.02456299e+01],   [ 7.88912339e-01, -4.74199063e+00],   [ 1.48140293e+01,  1.33348328e+01],   [ 9.27534981e+00,  1.09869664e+00],   [ 3.37781133e+00,  1.32140226e+01],   [-6.71011343e+00, -6.95749509e-01],   [ 1.27855930e+01, -1.97821403e+00],   [ 8.05128527e+00,  2.68310050e+00],   [ 7.10726998e+00,  3.93132733e+00],   [ 6.86029340e+00,  1.19676570e+01],   [ 1.14284097e+01,  1.20762558e+01],   [ 1.63071275e+01, -3.52246803e+00],   [ 6.98596358e-01,  3.31101867e+00],   [ 1.95937936e+00, -1.20840935e+00],   [ 9.64212934e+00,  3.36372544e+00],   [ 1.37125913e+01,  4.97564854e+00],   [-4.07754147e+00,  1.65273344e+00],   [-2.07724562e+00,  1.37082226e+01],   [ 8.94501494e+00, -8.41685724e+00],   [-2.10767190e+00,  1.68268990e+01],   [-2.86465322e+00,  1.28886368e+01],   [ 1.64457386e+01,  1.68710023e+01],   [-1.55768457e+00,  4.12364122e+00],   [-4.35916393e+00,  8.20232573e+00],   [ 6.62093513e+00,  1.50692854e+01],   [-2.60520807e+00, -4.96779953e+00],   [-5.89839635e-01, -8.07682047e+00],   [ 6.58910375e+00,  1.40801169e+01],   [ 6.30814360e+00, -1.13638846e+00],   [ 4.92108272e+00,  9.34233225e+00],   [ 1.29328981e+01,  1.67736062e+01]],   [[ 1.96757369e+00,  1.83502317e+01],   [ 1.18599296e+01, -6.39815727e+00],   [-5.42702022e+00,  8.33234889e+00],   [ 1.06441075e+00,  1.35062604e+01],   [ 1.45689026e+01,  8.96197511e+00],   [-2.12328575e+00,  1.28091299e+01],   [-3.62300999e+00,  1.50651712e+00],   [ 2.11139916e+01,  7.63979077e+00],   [ 1.21299259e+01, -1.70578650e+00],   [ 1.23760801e+01,  3.66873193e+00],   [-1.87166398e+00, -2.03301797e+00],   [ 2.10119098e+01, -6.58204855e-01],   [ 1.06286905e+01,  5.49973369e-01],   [ 7.24278111e+00,  1.21804942e+01],   [ 6.23824901e+00,  1.28431535e+01],   [ 1.41736038e+01,  7.14060926e+00],   [ 1.56410932e+01, -3.66059442e+00],   [ 1.38928287e-01,  4.39909504e+00],   [ 4.96731251e+00,  1.79082324e-01],   [ 1.18145486e+01,  1.12731267e-01],   [ 1.32798296e+01,  9.58655831e+00],   [-5.25577091e+00,  2.18410846e-01],   [-4.65377536e+00,  2.09654518e+01],   [ 5.98945803e+00,  5.94634271e-01],   [-8.75484313e+00,  1.91378670e+01],   [ 2.18623990e+00,  1.66859990e+01],   [ 5.87133899e+00,  1.61668579e+01],   [ 1.15419667e-01,  1.14218370e+00],   [-8.46323383e-01,  7.04352705e+00],   [ 1.44011019e+01,  1.39062071e+01],   [-1.04155558e+00, -1.52733744e+00],   [-3.61200232e+00, -7.09724056e+00],   [ 1.03238881e+01,  1.82488930e+01],   [ 1.40288301e+01, -5.07253230e+00],   [ 3.12139281e+00,  1.03067530e+01],   [ 1.24917583e+01,  1.97261437e+01]],   [[ 8.00998039e+00,  1.75169237e+01],   [ 1.15147363e+01, -3.95701865e+00],   [ 1.64120525e+00,  7.49282416e+00],   [ 1.02561444e+01,  1.04233822e+01],   [ 1.29531733e+01,  2.85534826e+00],   [-5.70843796e+00,  4.50453461e+00],   [-3.66579341e+00, -2.03388908e+00],   [ 1.59395011e+01,  1.15925955e+01],   [ 7.66303332e+00,  6.35571692e+00],   [ 1.36526288e+01,  4.26838514e+00],   [ 6.32203298e-01,  4.04556826e+00],   [ 2.16165137e+01,  4.00348255e+00],   [ 3.95766986e+00,  1.31949242e-01],   [ 1.13847190e+01,  6.94614766e+00],   [ 1.13331927e+01,  1.59138867e+01],   [ 9.04913329e+00,  7.29041183e+00],   [ 1.70381766e+01, -4.26491596e+00],   [ 3.95740170e+00,  6.72936441e+00],   [ 4.71471008e+00, -2.59523687e+00],   [ 9.18401463e+00, -8.33982431e-01],   [ 1.41669912e+01,  5.10344785e+00],   [-1.28813233e+00, -1.79982404e+00],   [-8.42410958e+00,  1.76831522e+01],   [ 5.68742588e+00, -8.08491015e+00],   [-1.94800720e+00,  2.19302755e+01],   [-1.03926343e+00,  1.64475749e+01],   [ 1.00388767e+01,  1.72827117e+01],   [ 2.15633956e-01,  7.14203291e+00],   [-4.19246809e+00,  4.50352482e+00],   [ 3.82096545e+00,  5.72508186e+00],   [-9.46208074e+00, -3.15010425e+00],   [-3.27894772e+00, -5.98954300e+00],   [ 6.28689156e+00,  1.36688355e+01],   [ 8.37458533e+00, -1.36942116e+00],   [ 8.77965159e+00,  8.76377766e+00],   [ 1.19219286e+01,  1.89998300e+01]],   [[ 6.39456402e+00,  1.69429518e+01],   [ 1.07366850e+01, -1.84532126e+00],   [ 1.10157096e+00, -2.39575125e+00],   [ 5.34245725e+00,  7.50437301e+00],   [ 1.82222557e+01,  5.45734301e+00],   [-6.94909893e-02,  1.10845081e+01],   [-4.47838440e+00, -4.33994665e-01],   [ 1.90426197e+01,  1.03543932e+01],   [ 1.12233822e+01,  5.86081817e+00],   [ 8.61865315e+00,  4.90829636e+00],   [-2.65344582e-02,  8.33401059e+00],   [ 1.72670990e+01, -4.33960231e-01],   [ 1.36368187e+01,  3.81439687e+00],   [ 1.03240031e+01,  3.00397908e+00],   [ 1.18946723e+01,  9.54684006e+00],   [ 1.07910484e+01,  7.99414927e+00],   [ 1.63315139e+01, -4.71201188e+00],   [ 8.04648872e+00, -1.08511235e+00],   [ 5.65500041e+00, -5.91172533e+00],   [ 1.38854428e+01, -1.14325997e+00],   [ 1.69404836e+01,  5.88260876e+00],   [ 2.12606461e+00, -3.12838819e+00],   [-5.68725582e+00,  1.70170617e+01],   [ 7.65938635e+00,  5.16150283e+00],   [-1.33377328e+00,  2.07710335e+01],   [-3.62283821e+00,  8.08325713e+00],   [ 9.93756584e+00,  1.28705253e+01],   [ 4.60692593e+00,  1.41280131e+00],   [-3.22637001e-01,  1.28161532e+01],   [ 8.10189911e+00,  1.21594340e+01],   [-4.22575785e+00, -1.59330882e+00],   [ 1.28015963e+00, -6.12500869e+00],   [ 1.01095446e+01,  2.15570757e+01],   [ 1.21499071e+01, -2.56408520e+00],   [ 6.23599251e+00,  4.38767339e+00],   [ 1.18034243e+01,  1.79870324e+01]],   [[-1.14313479e-01,  1.36511005e+01],   [ 1.66979951e+01,  1.96639375e+00],   [-5.50371446e+00,  1.26045323e+01],   [ 2.63589440e+00,  1.35087085e+01],   [ 1.57652507e+01,  9.36154327e+00],   [-8.92794227e+00,  1.28374371e+01],   [-4.66260585e+00,  1.83697035e+00],   [ 2.05699683e+01,  1.02700744e+01],   [ 1.27681240e+01,  4.80461194e+00],   [ 5.42490177e+00,  9.70726381e+00],   [ 1.04993072e+00,  5.63554211e+00],   [ 1.55593401e+01, -9.90314488e-01],   [ 6.92203451e+00,  5.48318463e+00],   [ 1.25234238e+01,  6.64823558e+00],   [ 6.45364969e+00,  2.94293010e+00],   [ 7.86779339e+00,  6.82196464e+00],   [ 2.04441497e+01, -5.27009413e+00],   [ 5.61742215e+00,  8.39954570e+00],   [ 3.75242487e+00, -4.88307194e-01],   [ 8.53688762e+00, -2.14118304e+00],   [ 1.54788658e+01,  4.97389647e+00],   [ 1.73022632e+00, -5.78114896e-02],   [-1.51711541e+00,  1.38803833e+01],   [ 4.75277703e+00,  1.89514192e+00],   [-4.56765901e+00,  1.67941122e+01],   [-5.63185213e-01,  1.36952666e+01],   [ 1.42948274e+01,  1.50816407e+01],   [-1.45773316e+00,  1.82780942e+00],   [-3.01799837e+00,  5.77636092e+00],   [ 6.77772796e+00,  3.07106818e+00],   [-8.41781711e+00, -3.65157449e+00],   [ 2.59099430e+00, -4.80088269e+00],   [ 5.03807832e+00,  1.16782706e+01],   [ 1.04475388e+01, -8.82097807e+00],   [ 9.21219321e+00,  7.79208449e+00],   [ 9.96922967e+00,  1.39098749e+01]],   [[ 1.92560798e+00,  1.36293917e+01],   [ 7.93391858e+00, -6.85285781e+00],   [-2.39976652e+00,  5.21133167e+00],   [ 6.87143302e+00,  1.38876533e+01],   [ 1.29829346e+01,  4.88639629e+00],   [-6.62542630e+00,  1.04462167e+01],   [ 1.09393911e+00,  8.29728791e-01],   [ 2.29189235e+01,  1.14601810e+01],   [ 8.69590223e+00,  6.07012052e+00],   [ 1.13684808e+01,  8.38710030e+00],   [-7.80668942e+00,  2.94943773e+00],   [ 1.89066949e+01, -4.85725874e+00],   [ 7.32387608e+00, -4.43932884e+00],   [ 5.10160467e+00,  3.64788025e+00],   [ 1.54044778e+01,  1.11568636e+01],   [ 8.56371924e+00,  3.96014973e+00],   [ 1.64627730e+01, -3.24459020e+00],   [ 8.27139188e+00,  5.85971729e+00],   [-1.18178478e+00, -2.02685912e+00],   [ 1.60594276e+01, -3.02658175e-01],   [ 1.96482421e+01,  2.13612574e+00],   [ 3.15351068e+00, -2.63170893e+00],   [-5.22880461e+00,  1.98358030e+01],   [ 5.41215455e+00, -1.52875091e+00],   [-5.41836901e-01,  2.00684645e+01],   [-1.50807855e+00,  1.15880647e+01],   [ 1.24033436e+01,  1.39721200e+01],   [ 5.05945036e+00,  2.52215843e-01],   [-5.64217452e+00,  4.93151492e+00],   [ 1.18888255e+00,  1.62549592e+01],   [-4.69453219e+00, -4.71242720e+00],   [-8.76693715e-01, -1.24399012e+00],   [ 5.46075420e+00,  1.82552518e+01],   [ 8.55501570e+00, -5.48119295e+00],   [ 7.63463216e+00,  1.07151181e+01],   [ 1.34269185e+01,  1.45331747e+01]],   [[ 5.19076116e-01,  1.95640264e+01],   [ 1.08263813e+01, -9.93439685e+00],   [ 4.22316138e+00,  3.12873862e+00],   [ 5.76939160e+00,  1.41362110e+01],   [ 1.70343583e+01,  4.09660584e+00],   [-2.68792083e+00,  6.12093799e+00],   [ 1.84874706e-01,  4.95567661e+00],   [ 1.84867574e+01,  3.88504758e+00],   [ 8.47259959e+00,  2.91563581e+00],   [ 9.49326939e+00,  4.58977510e+00],   [-5.90607653e+00,  5.29881309e+00],   [ 1.90089623e+01, -2.64256144e-01],   [ 1.12082492e+01, -8.08263023e+00],   [ 5.08598569e+00,  5.83364092e+00],   [ 9.90484666e+00,  1.29509383e+01],   [ 1.44996500e+01,  8.83725074e+00],   [ 1.40471323e+01, -8.02479177e+00],   [ 5.78233071e+00, -1.53429958e+00],   [ 5.41104256e+00, -6.33775803e-01],   [ 8.95724453e+00,  2.56976515e+00],   [ 1.41539482e+01,  5.49349107e+00],   [ 1.07641082e+00,  1.41834549e+00],   [-6.94006249e-01,  1.79322180e+01],   [ 9.01462238e+00, -4.24069482e+00],   [ 3.07950459e+00,  1.94905397e+01],   [-4.18692296e-01,  8.82954456e+00],   [ 1.17125257e+01,  1.25255403e+01],   [-3.17156630e+00,  3.46608645e+00],   [-6.61578880e+00,  5.16312907e+00],   [ 7.85759809e+00,  1.08024316e+01],   [-4.69926008e+00, -4.48990430e+00],   [ 2.45431609e+00, -4.65114388e+00],   [ 7.80615386e+00,  2.38991397e+01],   [ 1.05081016e+01, -1.46394925e-02],   [ 4.06745895e+00,  3.28002928e+00],   [ 1.25482684e+01,  1.61169679e+01]],   [[ 2.64566843e+00,  2.03004653e+01],   [ 1.22658971e+01, -2.62763934e+00],   [-3.82485093e-01,  6.08959554e+00],   [ 3.42467492e+00,  1.89764654e+01],   [ 1.36954231e+01,  8.36999948e+00],   [-7.08854235e+00,  9.94145465e+00],   [-1.66333284e+00,  1.06406719e+00],   [ 2.03812824e+01,  1.01130189e+01],   [ 8.91719495e+00,  5.56510871e+00],   [ 6.80183238e+00,  5.68181787e+00],   [-4.71392332e+00, -2.72944196e+00],   [ 1.69349743e+01,  2.31509362e+00],   [ 1.18333521e+01,  5.76660750e+00],   [ 4.27140071e+00,  8.48289084e+00],   [ 1.27441331e+01,  1.05732843e+01],   [ 1.25496495e+01,  8.81504501e+00],   [ 1.21609654e+01, -2.43075152e+00],   [ 1.11813425e+00,  2.21717441e+00],   [-2.26627854e+00, -1.53474616e+00],   [ 1.20318143e+01,  3.40017785e+00],   [ 1.41658891e+01,  7.32078508e-01],   [ 4.06909981e+00, -2.83310056e-01],   [-2.07295990e+00,  1.94163410e+01],   [ 1.03126474e+01,  2.10083152e+00],   [-2.26695219e+00,  1.86683938e+01],   [-2.10945800e+00,  1.56458190e+01],   [ 7.31479593e+00,  1.09484170e+01],   [ 2.53242989e+00,  1.06027745e+00],   [-2.23446977e+00,  9.40497951e+00],   [ 2.81209679e+00,  1.97222257e+00],   [-7.51611494e+00, -3.24384545e+00],   [-4.11189752e+00, -7.79820719e+00],   [ 3.89911359e+00,  1.66434708e+01],   [ 1.01823686e+01, -5.11665118e+00],   [ 1.02593450e+01,  1.23262178e+01],   [ 1.68395541e+01,  1.12731130e+01]],   [[ 7.22744506e-01,  1.73619003e+01],   [ 1.06578268e+01, -5.62725041e+00],   [ 2.60728574e+00,  9.07614276e+00],   [ 7.46024219e+00,  1.21988797e+01],   [ 2.04727344e+01,  4.19224585e+00],   [ 8.97785899e-01,  1.17295996e+01],   [-4.39583468e+00, -5.12639047e+00],   [ 1.47378788e+01,  9.46111514e+00],   [ 1.42971068e+01,  3.15977762e+00],   [ 6.58596844e+00,  8.33293201e+00],   [-5.05735971e+00,  1.59407161e+00],   [ 1.71446641e+01, -1.07865083e+00],   [ 1.09864858e+01, -1.81457332e+00],   [ 7.34559183e+00,  7.33077673e+00],   [ 1.02667374e+01,  1.18560007e+01],   [ 1.38726381e+01,  8.53345947e+00],   [ 1.75357423e+01, -5.21878374e+00],   [ 6.71794004e+00,  6.39586769e+00],   [ 6.52220972e+00, -2.33481793e+00],   [ 1.21301993e+01,  3.66478787e+00],   [ 1.22378008e+01,  4.89415984e+00],   [ 7.43268073e+00,  6.26370402e+00],   [ 3.03712996e-01,  1.93298496e+01],   [ 2.59082106e+00, -2.48024868e+00],   [-3.17413956e+00,  1.43370336e+01],   [-3.08998604e-01,  1.26545095e+01],   [ 9.33084872e+00,  1.49153434e+01],   [ 2.41117391e+00,  3.16573144e+00],   [-3.95522064e+00,  8.46830574e+00],   [ 5.82985681e+00,  6.23847490e+00],   [-3.06626389e+00, -4.89938673e+00],   [-6.60695336e+00, -6.97540091e+00],   [ 5.05879287e+00,  1.83155955e+01],   [ 1.15446756e+01, -2.02303995e+00],   [ 3.73839039e+00,  9.90339621e+00],   [ 1.60628437e+01,  1.56684154e+01]],   [[ 2.78250265e+00,  1.88974322e+01],   [ 1.11478329e+01, -3.12668597e+00],   [ 3.03416031e+00,  1.34984863e+01],   [ 8.56614979e+00,  1.64048104e+01],   [ 1.13617799e+01,  3.76560026e+00],   [-5.57722641e+00,  1.66226399e+01],   [-3.88610143e+00,  1.25314384e+00],   [ 1.60101876e+01,  7.31426220e+00],   [ 1.27923753e+01,  4.76901831e+00],   [ 3.98232957e+00,  8.79411850e+00],   [-1.09445197e+01,  1.54817008e+00],   [ 1.84012311e+01, -3.30309058e+00],   [ 1.00108086e+01, -4.02640717e+00],   [ 7.75648250e+00, -4.67242142e-01],   [ 8.78538304e+00,  7.01939194e+00],   [ 1.00234008e+01,  5.92194563e+00],   [ 1.38844689e+01, -6.86153183e-01],   [ 1.21649596e+00,  6.28463247e+00],   [ 8.49817971e+00,  4.01911975e-01],   [ 9.96336991e+00, -2.28937751e+00],   [ 1.19185057e+01,  7.83749781e+00],   [ 1.83476937e+00,  3.05410036e+00],   [-1.18132317e+00,  1.68070652e+01],   [ 7.31127756e+00,  1.76569086e+00],   [ 2.43818488e+00,  1.67271649e+01],   [ 4.43026038e+00,  1.41413674e+01],   [ 1.46553137e+01,  1.87809622e+01],   [-3.17406914e+00, -2.41186560e+00],   [-2.88267428e+00,  1.41311412e+01],   [ 5.51375248e+00,  9.97781189e+00],   [-7.32367763e+00,  4.28627816e-01],   [-7.53787916e+00, -1.28923166e+01],   [ 9.69860638e+00,  1.58458760e+01],   [ 8.37845067e+00, -4.54649935e+00],   [ 7.53954153e+00,  6.28146388e+00],   [ 1.15832410e+01,  1.40319101e+01]],   [[ 7.02525562e+00,  1.44150613e+01],   [ 8.26486809e+00, -3.76381582e+00],   [ 1.36143476e+00,  1.16631576e+01],   [ 3.99359199e+00,  1.14699686e+01],   [ 1.76228342e+01,  8.24409156e+00],   [-3.96705192e+00,  1.12538854e+01],   [-8.52051456e-01, -1.88927107e-01],   [ 1.64821475e+01,  7.03982040e+00],   [ 7.94693808e+00,  8.65479162e-01],   [ 9.98428775e+00,  5.74543023e+00],   [-1.05687126e+00,  2.82593970e+00],   [ 1.17110796e+01,  4.08971808e+00],   [ 8.84382391e+00,  4.13168551e+00],   [ 9.94152350e+00,  1.37705708e+01],   [ 1.07548455e+01,  1.47468771e+01],   [ 1.41126301e+01,  5.47955896e+00],   [ 1.54693451e+01, -7.83179876e+00],   [-8.76667910e-01,  4.67877135e+00],   [ 5.85366589e+00,  1.97976167e+00],   [ 1.37508134e+01,  4.39047763e+00],   [ 1.22982551e+01,  4.74739765e+00],   [ 3.67499607e+00,  1.46133546e+00],   [-4.28199870e+00,  2.05385014e+01],   [ 1.72830951e+00, -5.74131880e+00],   [-2.93469563e+00,  2.02666419e+01],   [ 1.30816185e+00,  8.29955729e+00],   [ 1.26871630e+01,  2.00962475e+01],   [ 2.24440499e+00,  4.66796649e+00],   [-1.06205796e+01,  5.26884326e+00],   [ 9.49109050e+00,  1.03990322e+01],   [-2.39588351e+00, -7.30052327e+00],   [-4.42765034e-01, -1.01288677e+01],   [ 2.33461519e+00,  1.70416051e+01],   [ 1.25596515e+01, -2.81764670e+00],   [ 5.73709261e+00,  2.65050393e+00],   [ 1.82336122e+01,  1.68252070e+01]],   [[ 1.04127606e+01,  1.90543006e+01],   [ 6.30473710e+00, -1.69925532e+00],   [-4.65173096e+00,  5.05887910e+00],   [ 2.51155874e+00,  1.32587071e+01],   [ 1.73811013e+01,  2.71001107e+00],   [-7.48283016e+00,  1.32942015e+01],   [-1.71656496e+00,  5.83394533e-01],   [ 8.75418206e+00,  9.71123649e+00],   [ 8.76820874e+00,  5.98219286e-01],   [ 1.08688523e+01,  1.46222382e+00],   [-7.80243822e+00,  2.36956079e+00],   [ 1.67515295e+01, -2.32517803e+00],   [ 1.28058767e+01,  4.65786476e+00],   [ 5.26327031e+00,  6.07453432e+00],   [ 8.98744066e+00,  6.56889179e+00],   [ 1.22520944e+01,  9.55485326e+00],   [ 2.23768555e+01, -6.04723428e+00],   [ 8.24660189e+00,  8.52342075e+00],   [ 3.97162444e+00,  1.86636067e+00],   [ 1.54370931e+01,  7.90402993e+00],   [ 1.56740066e+01,  5.41904918e+00],   [-1.39631798e+00,  1.05720278e+00],   [-3.78629418e-01,  1.71353087e+01],   [ 3.34764234e+00, -3.81942909e+00],   [-2.44693235e+00,  1.85217093e+01],   [-3.95460678e+00,  1.40304910e+01],   [ 1.29446324e+01,  1.80638742e+01],   [-1.05380296e-01, -3.00084518e+00],   [-4.75458394e+00,  1.14688095e+01],   [ 5.24665143e+00,  4.28198147e+00],   [-8.92002780e+00, -2.30295968e+00],   [ 9.30835958e-01, -2.93128907e+00],   [ 8.33794498e+00,  1.13618146e+01],   [ 7.30681445e+00, -1.00495658e+00],   [ 6.28572371e+00,  9.64496872e+00],   [ 1.64269452e+01,  1.63163470e+01]]], dtype=np.float64))), [Measurement(landmark_id='landmark-17', r=16.618152723564037, b=1.3142893395563815), Measurement(landmark_id='landmark-24', r=13.2895332911168, b=-0.31259629224640056), Measurement(landmark_id='landmark-9', r=7.464330626838889, b=1.6023514481303975), Measurement(landmark_id='landmark-16', r=13.953110685803654, b=1.3316941778460758), Measurement(landmark_id='landmark-33', r=17.184808584191327, b=0.3532187535248875), Measurement(landmark_id='landmark-4', r=12.025836178136165, b=1.8961532532226417), Measurement(landmark_id='landmark-1', r=1.6749692253611657, b=3.122499479032094), Measurement(landmark_id='landmark-34', r=18.754089052746934, b=0.5795987270710623), Measurement(landmark_id='landmark-26', r=12.104851657816209, b=0.29414815863056887), Measurement(landmark_id='landmark-20', r=9.355902446373292, b=0.5546240629649799), Measurement(landmark_id='landmark-2', r=5.5283195865483465, b=2.475440923538948), Measurement(landmark_id='landmark-31', r=15.334805326513386, b=0.03067504038519622), Measurement(landmark_id='landmark-19', r=8.903470181651809, b=0.00820412066646277), Measurement(landmark_id='landmark-21', r=10.570209249630953, b=0.7049624503229832), Measurement(landmark_id='landmark-28', r=16.512836922809804, b=0.8046138129043872), Measurement(landmark_id='landmark-27', r=13.412586918317222, b=0.569748591066251), Measurement(landmark_id='landmark-30', r=16.333197492556163, b=-0.19778098213865833), Measurement(landmark_id='landmark-14', r=5.871513371998722, b=0.8164102826198404), Measurement(landmark_id='landmark-13', r=3.3461713905816604, b=-0.011310923317063418), Measurement(landmark_id='landmark-25', r=12.78397547006133, b=0.13032645317965927), Measurement(landmark_id='landmark-32', r=15.233356970366522, b=0.37709343559395214), Measurement(landmark_id='landmark-7', r=1.3458754237823045, b=0.06818228212795595), Measurement(landmark_id='landmark-5', r=16.756878132799947, b=1.7883800563678198), Measurement(landmark_id='landmark-12', r=7.1148749330293395, b=-0.6143333543391841), Measurement(landmark_id='landmark-11', r=17.801634869866124, b=1.6855607802948347), Measurement(landmark_id='landmark-10', r=12.357380410658957, b=1.465350634062304), Measurement(landmark_id='landmark-29', r=19.995545866879915, b=0.9845834633711026), Measurement(landmark_id='landmark-8', r=2.7116385362504456, b=1.6052772312820927), Measurement(landmark_id='landmark-3', r=9.374135604821388, b=2.0951851221342133), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-0', r=5.22041900234916, b=-2.4582789646260825), Measurement(landmark_id='landmark-6', r=5.7393678771301335, b=-1.6140727018312917), Measurement(landmark_id='landmark-23', r=18.217513449458522, b=1.0845840064251768), Measurement(landmark_id='landmark-18', r=10.527744788801282, b=-0.34842437630350404), Measurement(landmark_id='landmark-15', r=9.577624532824075, b=0.9148562052065338), Measurement(landmark_id='landmark-35', r=22.67944597222941, b=0.8450610387947912)], np.array([0.00000000e+000, 9.88131292e-324, 4.45810892e-175, 3.56795095e-066, 5.79754745e-098, 1.02078810e-216, 0.00000000e+000, 4.15469530e-122, 7.88047520e-274, 2.51973479e-322, 1.37155049e-170, 0.00000000e+000, 1.32313709e-089, 0.00000000e+000, 5.47806515e-123, 1.52801009e-264, 1.00000000e+000, 5.03346026e-218, 1.82949973e-015, 0.00000000e+000], dtype=np.float64))
print('Tests passed.')

### 3.4 SLAM: Particle Filter Update

Please implement the update step of SLAM.

The procedure is similar to `Localization.update`, except that it does an additional `add_new_landmarks`
step after forward updating with the `motion_model` and before `compute_weights`.


For reference, our solution is **16** line(s) of code.

In addition to all the utilities defined at the top of this notebook, the following functions are available in this question environment: `add_new_landmarks`, `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(SLAM)
def update(self: SLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the particles, based on the command and measurements.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError() 

### Tests

In [ ]:
def slam_update_test(_, infer: SLAM, command: Command,
                     measurements: Sequence[Measurement], pose: Pose,
                     landmarks: Sequence[Landmark]):
    """Checks if estimation after one step update matchs our estimation."""
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    est_landmarks = infer.estimated_landmarks()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=1.0), "Estimated pose should be quite close to our estimation"

    landmarks = {l.id: l for l in landmarks}
    est_landmarks = {l.id: l for l in est_landmarks}

    assert landmarks.keys() == est_landmarks.keys()

    for lid in landmarks:
        assert np.allclose(
            [landmarks[lid].x, landmarks[lid].y],
            [est_landmarks[lid].x, est_landmarks[lid].y],
            atol=1.0
        ), "Estimated landmark location should be quite close to our estimation"

slam_update_test(update, SLAM(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 5.79948578e-09]], dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=100, particles=SLAMParticles(x=np.zeros((100,), dtype=np.float64), y=np.zeros((100,), dtype=np.float64), theta=np.zeros((100,), dtype=np.float64), landmarks_id_to_idx={}, landmarks_loc=np.array([], dtype=np.float64).reshape((100, 0, 2)))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='L-only', r=1.4143392925941884, b=0.7853881030367988)], Pose(x=1.0006045140443256, y=-0.00012787494016420898, theta=0.7839209541825823), [Landmark(id='L-only', x=1.0025957357392745, y=1.4142827160763445)])
print('Tests passed.')

#### Experiment 10

Let's visualize our SLAM particle filter.

Please generate an animation of the robot carrying out inference in the default Simulator. 

In particular:

- Run SLAM in the default Simulator with `num_particles=1000` and `max_sensing_range=6`. 
- Visualize the inference using `render_animation`.

<b>Submission Material 10:</b> 
In your submission, please include the generated animation. 
Name the file 'slam.mp4'.

#### Experiment 11

Let's further visualize our SLAM particle filter for more settings.

Please generate plots of the robot carrying out inference in the default Simulator. 

In particular:

- Run SLAM in the default Simulator, for all combinations of these parameters:
    - `num_particles` in `[100, 1000]`
    - `max_sensing_range` in `[6, np.inf]`
- Use `plot_simulation_result` to plot the obtained estimated trajectory and landmarks for each setting. 

<b>Submission Material 11:</b> 
In your submitted pdf, please include the generated figures.
For each plot, indicate its setting (i.e., the parameter values) in the caption or title.

## Final Submission
Your final submission to gradescope should include the following files:

- project02_release.ipynb: Your completed notebook with output from running each cell. Make sure to save. 
- localization.mp4: The animation of the successful run of the localizing robot. 
- slam.mp4: The animation of the successful run of the mapping robot. 
- A PDF of the results of the experiments. 

## Feedback

If you have any feedback for us, please complete [this form](https://forms.gle/15BmSw7snfzT46zG8)!